In [14]:
import numpy as np
import matplotlib.pyplot as plt
import argparse
from scipy.stats import gaussian_kde
import os
import yaml
import h5py
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import time

from mpi_master_slave import Master, Slave
from mpi_master_slave import WorkQueue
from mpi4py import MPI
import warnings
warnings.filterwarnings('ignore')

In [15]:
import sys
sys.path.append(r'../')
sys.path.append(r'../toollib_DisbaCode')
from objective_function import (ObjectiveFunctionDerivativeUsed, Forward)
from toollib_DisbaCode import forward
from toollib_DisbaCode import objective_function
from toollib_DisbaCode import empirical_relation
from toollib_DisbaCode import inversion
from toollib_DisbaCode import plot_disp
from toollib_DisbaCode import plot_model
from toollib_DisbaCode import plot_lcurve
from toollib_DisbaCode import plot_inversion    
from toollib_DisbaCode import plot_kernel

In [16]:
with open('a-project.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
proj_name = proj['name']
proj_name = 'project/output_FJSJ_12-01/'
proj_name

'project/output_FJSJ_12-01/'

In [17]:
filename = proj_name+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = proj_name+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [18]:
dir_ds = info_basic['dir_ds']
dir_image = info_basic['dir_image']
dir_inv_dispernet = info_basic['dir_inv_dispernet'] 
d_len = info_basic['d_len']
f = info_basic_bi['f']
c = np.linspace(info_basic['c_min'], info_basic['c_max'], info_basic['c_num'])

In [19]:
key_subworks = info_basic['key_subworks']
#key_subworks = ['11-04']
key_subworks

['11-04',
 '13-04',
 '15-04',
 '17-04',
 '19-04',
 '21-04',
 '23-04',
 '25-04',
 '27-04',
 '29-04',
 '31-04',
 '33-04',
 '35-04',
 '37-04',
 '39-04',
 '41-04',
 '43-04',
 '45-04',
 '47-04',
 '49-04',
 '11-05',
 '13-05',
 '15-05',
 '17-05',
 '19-05',
 '21-05',
 '23-05',
 '25-05',
 '27-05',
 '29-05',
 '31-05',
 '33-05',
 '35-05',
 '37-05',
 '39-05',
 '41-05',
 '43-05',
 '45-05',
 '47-05',
 '49-05',
 '11-06',
 '13-06',
 '15-06',
 '17-06',
 '19-06',
 '21-06',
 '23-06',
 '25-06',
 '27-06',
 '29-06',
 '31-06',
 '33-06',
 '35-06',
 '37-06',
 '39-06',
 '41-06',
 '43-06',
 '45-06',
 '47-06',
 '49-06',
 '11-07',
 '13-07',
 '15-07',
 '17-07',
 '19-07',
 '21-07',
 '23-07',
 '25-07',
 '27-07',
 '29-07',
 '31-07',
 '33-07',
 '35-07',
 '37-07',
 '39-07',
 '41-07',
 '43-07',
 '45-07',
 '47-07',
 '49-07',
 '11-08',
 '13-08',
 '15-08',
 '17-08',
 '19-08',
 '21-08',
 '23-08',
 '25-08',
 '27-08',
 '29-08',
 '31-08',
 '33-08',
 '35-08',
 '37-08',
 '39-08',
 '41-08',
 '43-08',
 '45-08',
 '47-08',
 '49-08',


### Forard and plot estimated disp and Vs

In [20]:
flag_forward = 0

In [21]:
def plot_fj0(ax,ds_linear,title0,f,c,xlim,index,cl):
    ax.imshow(np.flip(ds_linear,0),aspect='auto',extent=[min(f),max(f),min(c),max(c)],vmin=0.05, cmap = 'jet',clim=cl)
    #plt.imshow(np.flip(ds_linear,0),extent=[min(f),max(f),min(c),max(c)],aspect='auto',cmap='jet',vmin=0,vmax=1)
    #ax.pcolormesh(f,c,ds_linear,cmap='jet',vmin=0,vmax=1)
    if index == 0:
        ax.set_title(title0)
    else:
        ax.set_title('('+chr(96+index)+')',loc='left')
    ax.set_xlabel('Normalized Frequency/ Hz')
    ax.set_ylabel('Velocity/ m/s')
    #ax.set_xlim(xlim)
    return ax

In [22]:
def plot_fj(ax,ds_linear,title0,f,c,xlim,index,v_min,v_max,c_map):
    ax.imshow(np.flip(ds_linear,0),aspect='auto',extent=[min(f),max(f),min(c),max(c)],vmin=v_min,vmax = v_max, cmap = c_map)
    #plt.imshow(np.flip(ds_linear,0),extent=[min(f),max(f),min(c),max(c)],aspect='auto',cmap='jet',vmin=0,vmax=1)
    #ax.pcolormesh(f,c,ds_linear,cmap='jet',vmin=0,vmax=1)
    if index == 0:
        ax.set_title(title0)
    else:
        ax.set_title('('+chr(96+index)+')',loc='left')
    ax.set_xlabel('Normalized Frequency/ Hz')
    ax.set_ylabel('Velocity/ m/s')
    #ax.set_xlim(xlim)
    return ax

In [23]:
"""
#c_map = info_basic['c_map']
#v_min = info_basic['v_min']
#v_max = info_basic['v_max']
c_map = 'jet'
v_max = None
v_min = 0.1


os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/'+proj_name+'inv_dispernet')
file_config = 'config_inv.yml'
with open(file_config, 'r') as fp:
    config = yaml.safe_load(fp)

all_disp = 0
show_init = 0
xlim_f = [1,15]
ylim_c = [0.25,2.5]
clim = [0,1]

flag_num = 0
for key_subwork in key_subworks:
    fig,ax = plt.subplots(1,3,figsize=(27,8))

    os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/'+proj_name+'inv_dispernet/')
    data_file = 'data/ds_'+key_subwork+'curve.txt'
    file_model = 'model_'+key_subwork+'.txt'

    file_inv = 'inversion_files/ds_'+key_subwork+'curve/'
    if not os.path.exists(file_inv):
        print(key_subwork+'data file not exist')
        plt.close()
        continue



    disp_data = np.loadtxt(data_file)
    data_name = 'ds_'+key_subwork+'curve'
    ax[1] = plot_inversion.plot_disp(config, data_name, all_disp,ax[1])
    # s wave velocity inversion
    
    file_init = config['model_init']+key_subwork+'.txt'
    ax[2] = plot_inversion.plot_model(config, data_name, show_init, file_model,file_init,ax[2])
    

    # forward
    if flag_forward == 1:
        os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/'+proj_name+'inv_dispernet/')
        max_mode = 3
        filename = 'disp_model_'+key_subwork+'.txt'
        forward.forward_cal(info_basic,max_mode,file_model,filename)


    disp_model = np.loadtxt(filename)


    os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]
    title0 = "Linear stack dispersion curve "+str(d_len)+' days'
    ax[0].set_xlim(xlim_f)
    ax[0].set_ylim(ylim_c)
    #ax[0] = plot_fj(ax[0],ds_remove,title0,f,c,xlim_f,0,clim)
    ax[0] = plot_fj(ax[0],ds_remove,title0,f,c,xlim_f,0,v_min,v_max,c_map)
    
    

    
    modes = set(disp_data[:, 2].astype(int))
    for m in modes:
        d = disp_data[disp_data[:, 2] == m]
        if m==0:
            ax[0].plot(d[:, 0], d[:, 1], 'g^',label='data')
        else:
            ax[0].plot(d[:, 0], d[:, 1], 'g^')

    modes = set(disp_model[:, 2].astype(int))
    for m in modes:
        d = disp_model[disp_model[:, 2] == m]
        if m==0:
            ax[0].plot(d[:, 0], d[:, 1], 'r--',label='estemate',linewidth=5)
        else:
            ax[0].plot(d[:, 0], d[:, 1], 'r--',linewidth=5)
    ax[0].legend()
    ax[2].set_title(key_subwork)


    plt.tight_layout()
    plt.savefig(dir_image+'inversion_'+key_subwork+'.png',dpi=60)
    plt.close()
    flag_num += 1
    print('finish '+str(flag_num)+'/'+str(len(key_subworks)))
"""

'\n#c_map = info_basic[\'c_map\']\n#v_min = info_basic[\'v_min\']\n#v_max = info_basic[\'v_max\']\nc_map = \'jet\'\nv_max = None\nv_min = 0.1\n\n\nos.chdir(\'/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/\'+proj_name+\'inv_dispernet\')\nfile_config = \'config_inv.yml\'\nwith open(file_config, \'r\') as fp:\n    config = yaml.safe_load(fp)\n\nall_disp = 0\nshow_init = 0\nxlim_f = [1,15]\nylim_c = [0.25,2.5]\nclim = [0,1]\n\nflag_num = 0\nfor key_subwork in key_subworks:\n    fig,ax = plt.subplots(1,3,figsize=(27,8))\n\n    os.chdir(\'/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/\'+proj_name+\'inv_dispernet/\')\n    data_file = \'data/ds_\'+key_subwork+\'curve.txt\'\n    file_model = \'model_\'+key_subwork+\'.txt\'\n\n    file_inv = \'inversion_files/ds_\'+key_subwork+\'curve/\'\n    if not os.path.exists(file_inv):\n        print(key_subwork+\'data file not exist\')\n        plt.close()\n        continue\n\n\n\n    disp_data = np.loadtxt(data_file)\n    data_name = \'ds_\'+

In [24]:
"""
os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')
#key_subworks = ['12-04']
for key_subwork in key_subworks:
    fmin = 1
    fmax = 15
    cmin = 0.1
    cmax = 2.5
    vmax = 0.3
    
    file_disp = info_basic['dir_inv_dispernet']+'data/ds_'+key_subwork+'curve.txt'
    file_model = info_basic['dir_inv_dispernet']+'model_'+key_subwork+'.txt'
    disp = np.loadtxt(file_disp)
    m = int(max(set(disp[:,2])))

    fig,ax = plt.subplots(1,m+1,figsize=(7*(m+1),6))
    for mode in range(m+1):
        ax[mode] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,mode,vmax,ax[mode])
    #ax[0][1] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,1,vmax,ax[0][1])
    #ax[1][0] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,2,vmax,ax[1][0])
    #ax[1][1] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,3,vmax,ax[1][1])
    plt.tight_layout()
    plt.savefig(dir_image+'Sens-kernel_'+key_subwork+'.png',dpi=60)
    plt.close()
"""

"\nos.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')\n#key_subworks = ['12-04']\nfor key_subwork in key_subworks:\n    fmin = 1\n    fmax = 15\n    cmin = 0.1\n    cmax = 2.5\n    vmax = 0.3\n    \n    file_disp = info_basic['dir_inv_dispernet']+'data/ds_'+key_subwork+'curve.txt'\n    file_model = info_basic['dir_inv_dispernet']+'model_'+key_subwork+'.txt'\n    disp = np.loadtxt(file_disp)\n    m = int(max(set(disp[:,2])))\n\n    fig,ax = plt.subplots(1,m+1,figsize=(7*(m+1),6))\n    for mode in range(m+1):\n        ax[mode] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,mode,vmax,ax[mode])\n    #ax[0][1] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,1,vmax,ax[0][1])\n    #ax[1][0] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,2,vmax,ax[1][0])\n    #ax[1][1] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,3,vmax,ax[1][1])\n    plt.tight_layout()\n    plt.savefig(dir_image+'Sens-kernel_'+key_s

In [25]:
#key_subworks = ['35-16']
c_map = 'jet'
v_max = None
v_min = 0.1

os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/'+proj_name+'inv_dispernet')
file_config = 'config_inv.yml'
with open(file_config, 'r') as fp:
    config = yaml.safe_load(fp)

all_disp = 0
show_init = 0
xlim_f = [1,15]
ylim_c = [0.25,2.5]
clim = [0,1]

flag_num = 0
for key_subwork in key_subworks:
    print(key_subwork)
    fig,ax = plt.subplots(2,3,figsize=(27,16))

    os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/'+proj_name+'inv_dispernet/')
    data_file = 'data/ds_'+key_subwork+'curve.txt'
    file_model = 'model_'+key_subwork+'.txt'

    file_inv = 'inversion_files/ds_'+key_subwork+'curve/'
    if not os.path.exists(file_inv):
        print(key_subwork+'data file not exist')
        plt.close()
        continue



    disp_data = np.loadtxt(data_file)
    data_name = 'ds_'+key_subwork+'curve'
    ax[0][1] = plot_inversion.plot_disp(config, data_name, all_disp,ax[0][1])
    # s wave velocity inversion
    
    file_init = config['model_init']+key_subwork+'.txt'
    ax[0][2] = plot_inversion.plot_model(config, data_name, show_init, file_model,file_init,ax[0][2])
    

    # forward
    os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/'+proj_name+'inv_dispernet/')
    filename = 'disp_model_'+key_subwork+'.txt'
    if flag_forward == 1:
        
        max_mode = 3
        
        forward.forward_cal(info_basic,max_mode,file_model,filename)


    disp_model = np.loadtxt(filename)


    os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]
    title0 = "Linear stack dispersion curve "+str(d_len)+' days'
    ax[0][0].set_xlim(xlim_f)
    ax[0][0].set_ylim(ylim_c)
    #ax[0] = plot_fj(ax[0],ds_remove,title0,f,c,xlim_f,0,clim)
    ax[0][0] = plot_fj(ax[0][0],ds_remove,title0,f,c,xlim_f,0,v_min,v_max,c_map)
    
    

    
    modes = set(disp_data[:, 2].astype(int))
    for m in modes:
        d = disp_data[disp_data[:, 2] == m]
        if m==0:
            ax[0][0].plot(d[:, 0], d[:, 1], 'g^',label='data')
        else:
            ax[0][0].plot(d[:, 0], d[:, 1], 'g^')

    modes = set(disp_model[:, 2].astype(int))
    for m in modes:
        d = disp_model[disp_model[:, 2] == m]
        if m==0:
            ax[0][0].plot(d[:, 0], d[:, 1], 'r--',label='estemate',linewidth=5)
        else:
            ax[0][0].plot(d[:, 0], d[:, 1], 'r--',linewidth=5)
    ax[0][0].legend()
    ax[0][2].set_title(key_subwork)


    #plt.tight_layout()
    #plt.savefig(dir_image+'inversion_'+key_subwork+'.png',dpi=60)
    #plt.close()
    flag_num += 1
    

    os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')
    fmin = 1
    fmax = 15
    cmin = 0.1
    cmax = 2.5
    vmax = 0.3
    
    file_disp = info_basic['dir_inv_dispernet']+'data/ds_'+key_subwork+'curve.txt'
    file_model = info_basic['dir_inv_dispernet']+'model_'+key_subwork+'.txt'
    disp = np.loadtxt(file_disp)
    m = int(max(set(disp[:,2])))
    
    #fig,ax = plt.subplots(1,m+1,figsize=(7*(m+1),6))
    if m > 2:
        m = 2
    for mode in range(m+1):
        print(mode)
        try:
            ax[1][mode] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,mode,vmax,ax[1][mode])
        except:
            print('error')
    #ax[0][1] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,1,vmax,ax[0][1])
    #ax[1][0] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,2,vmax,ax[1][0])
    #ax[1][1] = plot_kernel.kernel_plot(fmin,fmax,cmin,cmax,file_disp,file_model,3,vmax,ax[1][1])

    plt.tight_layout()
    #plt.savefig(dir_image+'Sens-kernel_'+key_subwork+'.png',dpi=60)
    plt.savefig(dir_image+'invplot_all_'+key_subwork+'.png',dpi=100)
    

    print('finish '+str(flag_num)+'/'+str(len(key_subworks)))
    plt.close()

11-04
0
1
2


finish 1/280
13-04
0
1
2
finish 2/280
15-04
0
1
2
finish 3/280
17-04
0
1
2
finish 4/280
19-04
0
1
finish 5/280
21-04
0
1
finish 6/280
23-04
0
1
2
finish 7/280
25-04
0
1
2
finish 8/280
27-04
0
1
2
finish 9/280
29-04
0
1
2
finish 10/280
31-04
0
1
2
finish 11/280
33-04
0
1
2
finish 12/280
35-04
0
1
2
finish 13/280
37-04
0
1
2
finish 14/280
39-04
0
1
2
finish 15/280
41-04
0
1
2
finish 16/280
43-04
0
1
2
finish 17/280
45-04
0
1
2
finish 18/280
47-04
0
1
2
finish 19/280
49-04
0
1
2
finish 20/280
11-05
0
1
2
finish 21/280
13-05
0
1
2
finish 22/280
15-05
0
1
2
finish 23/280
17-05
0
1
2
finish 24/280
19-05
0
1
2
finish 25/280
21-05
0
1
2
finish 26/280
23-05
0
1
2
finish 27/280
25-05
0
1
2
finish 28/280
27-05
0
1
2
finish 29/280
29-05
0
1
2
finish 30/280
31-05
0
1
2
finish 31/280
33-05
0
1
2
finish 32/280
35-05
0
1
2
finish 33/280
37-05
0
1
error
2
finish 34/280
39-05
0
1
2
finish 35/280
41-05
0
1
2
finish 36/280
43-05
0
1
2
finish 37/280
45-05
0
1
2
finish 38/280
47-05
0
1
2
finish 39/280
49-05

### Write into xlsx

In [26]:
"""
# disp model xlsx
os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')
filename = 'disp_model.xlsx'

if not os.path.exists(info_basic['dir_inv_dispernet']+filename):
    df = pd.DataFrame()
    df.to_excel(info_basic['dir_inv_dispernet']+filename,index=False)

books = load_workbook(info_basic['dir_inv_dispernet']+filename)
writers = pd.ExcelWriter(info_basic['dir_inv_dispernet']+filename, engine='openpyxl')
writers.book = books

flag_num = 0
for key_subwork in key_subworks:
    file_model = dir_inv_dispernet + 'disp_model_'+key_subwork+'.txt'
    disp_model = np.loadtxt(file_model)
    
    
    df = pd.DataFrame()
    df['f'] = disp_model[:, 0]
    df['c'] = disp_model[:, 1]
    df['mode'] = disp_model[:, 2]

    df.to_excel(writers,key_subwork,index=False)

    flag_num += 1
    print('finish '+str(flag_num)+'/'+str(len(key_subworks)))

writers.save()
"""

"\n# disp model xlsx\nos.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')\nfilename = 'disp_model.xlsx'\n\nif not os.path.exists(info_basic['dir_inv_dispernet']+filename):\n    df = pd.DataFrame()\n    df.to_excel(info_basic['dir_inv_dispernet']+filename,index=False)\n\nbooks = load_workbook(info_basic['dir_inv_dispernet']+filename)\nwriters = pd.ExcelWriter(info_basic['dir_inv_dispernet']+filename, engine='openpyxl')\nwriters.book = books\n\nflag_num = 0\nfor key_subwork in key_subworks:\n    file_model = dir_inv_dispernet + 'disp_model_'+key_subwork+'.txt'\n    disp_model = np.loadtxt(file_model)\n    \n    \n    df = pd.DataFrame()\n    df['f'] = disp_model[:, 0]\n    df['c'] = disp_model[:, 1]\n    df['mode'] = disp_model[:, 2]\n\n    df.to_excel(writers,key_subwork,index=False)\n\n    flag_num += 1\n    print('finish '+str(flag_num)+'/'+str(len(key_subworks)))\n\nwriters.save()\n"

In [27]:
"""
# disp data xlsx
os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')
filename = 'structure_model.xlsx'

if not os.path.exists(info_basic['dir_inv_dispernet']+filename):
    df = pd.DataFrame()
    df.to_excel(info_basic['dir_inv_dispernet']+filename,index=False)

books = load_workbook(info_basic['dir_inv_dispernet']+filename)
writers = pd.ExcelWriter(info_basic['dir_inv_dispernet']+filename, engine='openpyxl')
writers.book = books

flag_num = 0
for key_subwork in key_subworks:
    file_model = dir_inv_dispernet + 'model_'+key_subwork+'.txt'
    model = np.loadtxt(file_model)
    
    
    df = pd.DataFrame()
    df['layer'] = model[:, 0]
    df['z'] = model[:, 1]
    df['rho'] = model[:, 2]
    df['vs'] = model[:, 3]
    df['vp'] = model[:, 4]
    #df['std'] = model[:, 5]
    df.to_excel(writers,key_subwork,index=False)

    flag_num += 1
    print('finish '+str(flag_num)+'/'+str(len(key_subworks)))

writers.save()
"""

"\n# disp data xlsx\nos.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')\nfilename = 'structure_model.xlsx'\n\nif not os.path.exists(info_basic['dir_inv_dispernet']+filename):\n    df = pd.DataFrame()\n    df.to_excel(info_basic['dir_inv_dispernet']+filename,index=False)\n\nbooks = load_workbook(info_basic['dir_inv_dispernet']+filename)\nwriters = pd.ExcelWriter(info_basic['dir_inv_dispernet']+filename, engine='openpyxl')\nwriters.book = books\n\nflag_num = 0\nfor key_subwork in key_subworks:\n    file_model = dir_inv_dispernet + 'model_'+key_subwork+'.txt'\n    model = np.loadtxt(file_model)\n    \n    \n    df = pd.DataFrame()\n    df['layer'] = model[:, 0]\n    df['z'] = model[:, 1]\n    df['rho'] = model[:, 2]\n    df['vs'] = model[:, 3]\n    df['vp'] = model[:, 4]\n    #df['std'] = model[:, 5]\n    df.to_excel(writers,key_subwork,index=False)\n\n    flag_num += 1\n    print('finish '+str(flag_num)+'/'+str(len(key_subworks)))\n\nwriters.save()\n"

In [28]:
"""
# disp data xlsx
os.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')
filename = 'disp_data.xlsx'

if not os.path.exists(info_basic['dir_inv_dispernet']+filename):
    df = pd.DataFrame()
    df.to_excel(info_basic['dir_inv_dispernet']+filename,index=False)

books = load_workbook(info_basic['dir_inv_dispernet']+filename)
writers = pd.ExcelWriter(info_basic['dir_inv_dispernet']+filename, engine='openpyxl')
writers.book = books

flag_num = 0
for key_subwork in key_subworks:
    file_model = dir_inv_dispernet + '/data/ds_'+key_subwork+'curve.txt'
    disp_model = np.loadtxt(file_model)
    
    
    df = pd.DataFrame()
    df['f'] = disp_model[:, 0]
    df['c'] = disp_model[:, 1]
    df['mode'] = disp_model[:, 2]

    df.to_excel(writers,key_subwork,index=False)

    flag_num += 1
    print('finish '+str(flag_num)+'/'+str(len(key_subworks)))

writers.save()
"""

"\n# disp data xlsx\nos.chdir('/home/harmon/data/git_repo/F-J_Workflow/San_Jasinto/')\nfilename = 'disp_data.xlsx'\n\nif not os.path.exists(info_basic['dir_inv_dispernet']+filename):\n    df = pd.DataFrame()\n    df.to_excel(info_basic['dir_inv_dispernet']+filename,index=False)\n\nbooks = load_workbook(info_basic['dir_inv_dispernet']+filename)\nwriters = pd.ExcelWriter(info_basic['dir_inv_dispernet']+filename, engine='openpyxl')\nwriters.book = books\n\nflag_num = 0\nfor key_subwork in key_subworks:\n    file_model = dir_inv_dispernet + '/data/ds_'+key_subwork+'curve.txt'\n    disp_model = np.loadtxt(file_model)\n    \n    \n    df = pd.DataFrame()\n    df['f'] = disp_model[:, 0]\n    df['c'] = disp_model[:, 1]\n    df['mode'] = disp_model[:, 2]\n\n    df.to_excel(writers,key_subwork,index=False)\n\n    flag_num += 1\n    print('finish '+str(flag_num)+'/'+str(len(key_subworks)))\n\nwriters.save()\n"

In [29]:
"""
dir_inv_dispernet = info_basic['dir_inv_dispernet']
os.listdir(dir_inv_dispernet)[0][-3:]
for file in os.listdir(dir_inv_dispernet):
    if file[-3:] == 'txt':
        os.remove(dir_inv_dispernet+file)
"""

"\ndir_inv_dispernet = info_basic['dir_inv_dispernet']\nos.listdir(dir_inv_dispernet)[0][-3:]\nfor file in os.listdir(dir_inv_dispernet):\n    if file[-3:] == 'txt':\n        os.remove(dir_inv_dispernet+file)\n"